In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip  # Import VideoFileClip from moviepy.editor
%matplotlib inline
import math

def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)   
    
    if len(img.shape) > 2:
        channel_count = img.shape[2] 
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def slope_lines(image, lines):
    img = image.copy()
    poly_vertices = []
    left_lines = [] 
    right_lines = []

    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                pass
            else:
                m = (y2 - y1) / (x2 - x1)
                c = y1 - m * x1

                if m < 0:
                    left_lines.append((m, c))
                elif m >= 0:
                    right_lines.append((m, c))

    if left_lines:
        left_line = np.mean(left_lines, axis=0)
        rows, cols = image.shape[:2]
        y1 = int(rows)
        y2 = int(rows * 0.6)
        x1 = int((y1 - left_line[1]) / left_line[0])
        x2 = int((y2 - left_line[1]) / left_line[0])
        poly_vertices.append((x1, y1))
        poly_vertices.append((x2, y2))

    if right_lines:
        right_line = np.mean(right_lines, axis=0)
        rows, cols = image.shape[:2]
        y1 = int(rows)
        y2 = int(rows * 0.6)
        
        # Avoid division by zero
        if right_line[0] != 0:
            x1 = int((y1 - right_line[1]) / right_line[0])
            x2 = int((y2 - right_line[1]) / right_line[0])
            
            poly_vertices.append((x2, y2))
            poly_vertices.append((x1, y1))

    cv2.fillPoly(img, pts=np.array([poly_vertices], 'int32'), color=(0, 255, 0))
    return cv2.addWeighted(image, 0.7, img, 0.4, 0.)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    line_img = slope_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.1, β=1., γ=0.):
    lines_edges = cv2.addWeighted(initial_img, α, img, β, γ)
    return lines_edges

def get_vertices(image):
    rows, cols = image.shape[:2]
    bottom_left = [cols * 0.15, rows]
    top_left = [cols * 0.45, rows * 0.6]
    bottom_right = [cols * 0.95, rows]
    top_right = [cols * 0.55, rows * 0.6] 

    ver = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    return ver

def lane_finding_pipeline(image):  
    gray_img = grayscale(image)
    smoothed_img = gaussian_blur(img=gray_img, kernel_size=5)
    canny_img = canny(img=smoothed_img, low_threshold=180, high_threshold=240)
    masked_img = region_of_interest(img=canny_img, vertices=get_vertices(image))
    houghed_lines = hough_lines(img=masked_img, rho=1, theta=np.pi/180, threshold=20, min_line_len=20, max_line_gap=180)
    output = weighted_img(img=houghed_lines, initial_img=image, α=0.8, β=1., γ=0.)
    return output

# Process video
def process_video(input_path, output_path):
    clip = VideoFileClip(input_path)
    processed_clip = clip.fl_image(lane_finding_pipeline)
    processed_clip.write_videofile(output_path, audio=False)

# Specify input and output paths
input_video_path = r"C:\Users\ILiYA\Desktop\Project\Media\Lane Line's Video\challenge_video.mp4"  # Change to your input video file
output_video_path = r"C:\Users\ILiYA\Desktop\Project\Media\video.mp4"  # Change to your desired output video file

# Process the video
process_video(input_video_path, output_video_path)


Moviepy - Building video C:\Users\ILiYA\Desktop\Project\Media\video.mp4.
Moviepy - Writing video C:\Users\ILiYA\Desktop\Project\Media\video.mp4



Moviepy - Done !
Moviepy - video ready C:\Users\ILiYA\Desktop\Project\Media\video.mp4
